In [ ]:
!pip install streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.4 MB/s eta 0:00:00


In [11]:
%%writefile app.py
import streamlit as st
import numpy as np
import tensorflow as tf
import cv2
from PIL import Image
from tensorflow.keras.models import load_model

# =============================
# Load Models
# =============================
classifier_model = tf.keras.models.load_model("/content/drive/MyDrive/brain_tumor_best_model(94%).h5")
segmentation_model = load_model("/content/drive/MyDrive/final_unet.keras",
                                custom_objects={'dice_loss': lambda y_true,y_pred: 0,
                                                'iou_coef': lambda y_true,y_pred: 0,
                                                'dice_coef': lambda y_true,y_pred: 0})

# Class labels
class_names = ["No Tumor is Existed", "Yes, Tumor is Existed"]

# =============================
# Streamlit UI
# =============================
st.title("🧠 Brain Tumor AI App")

mode = st.radio("Choose Task:", ["Tumor Classification", "Tumor Segmentation"])

uploaded_file = st.file_uploader("Upload MRI Image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:

    # =============================
    # Classification
    # =============================
    if mode == "Tumor Classification":
        image = Image.open(uploaded_file).convert("RGB")
        st.image(image, caption="Uploaded Image", use_column_width=True)
        img = image.resize((224,224))
        img_array = np.array(img)
        img_array = np.expand_dims(img_array, axis=0)

        predictions = classifier_model.predict(img_array)
        class_idx = np.argmax(predictions)
        confidence = np.max(predictions)

        st.subheader("🔍 Classification Result")
        st.write(f"**Prediction:** {class_names[class_idx]}")
        st.write(f"**Confidence:** {confidence:.2f}")

    # =============================
    # Segmentation
    # =============================
    else:
      # Read image
        file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
        img = cv2.imdecode(file_bytes, 1)
        img_resized = cv2.resize(img, (256, 256))
        img_norm = img_resized / 255.0
        img_input = np.expand_dims(img_norm, axis=0)

        # Predict mask
        pred_mask = segmentation_model.predict(img_input)[0]
        pred_mask = (pred_mask > 0.5).astype(np.uint8) * 255
        overlay = img_resized.copy()
        overlay[pred_mask[:,:,0] == 1] = [255, 0, 0]  # red overlay

        st.subheader("🖼️ Segmentation Results")
        col1, col2, col3 = st.columns(3)
        with col1:
            st.image(img_resized, channels="BGR", caption="Original MRI")
        with col2:
            st.image(pred_mask, clamp=True, caption="Predicted Mask")
        with col3:
            st.image(overlay, channels="BGR", caption="Overlay (Tumor in Red)")


Overwriting app.py


In [12]:
from pyngrok import ngrok

!ngrok authtoken 33SvwSeEUGtt5j4VhProgiHHVSR_7UstuxKcopDM3SjrzUDvz

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [13]:
from pyngrok import ngrok

# Kill any previous tunnels
ngrok.kill()

# Start streamlit
get_ipython().system_raw('streamlit run app.py --server.port 8501 &')

# Expose port 8501 via ngrok
public_url = ngrok.connect(8501)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://imagistically-deepwater-manual.ngrok-free.dev" -> "http://localhost:8501"
